In [1]:
import tensorflow as tf
import keras as kr
from keras.models import Sequential
from tensorflow.keras import layers

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *

In [4]:
import random

In [5]:
from deap import creator, base, tools, algorithms

Загрузка данных

In [6]:
data=pd.read_csv('C:/Users/Vi/Desktop/data.txt')

In [7]:
data=data.drop_duplicates()

функция оценки решения по MSE на валидационной выборке
и количеству нейронов ИНС

In [41]:
def evaluate_MO(individual):
    
    layer_number=individual[0][0]
    layer_size=individual[0][1]
    
    model = Sequential()
    model.add(layers.Dense(layer_size,input_dim=11, activation='sigmoid'))
    
    for i in range(layer_number):
        model.add(layers.Dense(layer_size, activation='sigmoid'))
        
    model.add(layers.Dense(1))
    
    model.compile(optimizer=kr.optimizers.Adam(learning_rate=0.001,beta_1=0.9,
    beta_2=0.999,epsilon=1e-07),loss='mean_squared_error')
    
    model.fit(x_train, y_train,epochs=50, batch_size=10, verbose=0)
    
    pred = model.predict(x_testval).flatten() 

    cost_f1 = mean_squared_error(y_testval, pred)
    
    cost_f2 = layer_number*layer_size
    
    return (cost_f1,cost_f2)

описание индивида
x1=количество слоев ИНС
x2=размер скрытого слоя

In [9]:
def gen():
    
    x1=random.randint(2, 6)
    x2=random.randint(10, 44)
        
    return [x1,x2]

функция мутации

In [10]:
def mutation(individual):
    
    print(individual)
    
    start=individual[0][1]
    start=start-5
    finish=individual[0][1]+5
    finish=finish+5
    individual[0][1]=random.randint(start, finish)

    return individual,

Заполнение пропусков и удаление повторов

In [11]:
data=data.fillna(value=0)

выделение обучающей и тестовой выборки

In [12]:
X = data[['скор ветра', 'выс волны', 'угол волны', 'скорость течения', 'направление течения', 'глубина', 'mass',
       'speedafloat', 'draft', 'owerallwidth', 'length']]

Y = data[['speed']]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.95, random_state=20)

нормализайия данных

In [14]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train -= mean
x_train /= std
x_test -= mean
x_test /= std

выдеоение валидационной выборки

In [15]:
x_trval, x_testval, y_trval, y_testval = train_test_split(x_test, y_test, test_size=0.5, random_state=20)

выбор параметров эволюционного алгоритма

In [42]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("gen", gen)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.gen, n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)



toolbox.register("evaluate", evaluate_MO)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selTournament)

C:\Users\Vi\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Vi\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


создание популяции

In [49]:
pop = toolbox.population(n=10)

In [50]:
print(pop)

[[[4, 38]], [[4, 34]], [[5, 33]], [[2, 25]], [[5, 16]], [[6, 40]], [[2, 13]], [[2, 36]], [[4, 44]], [[4, 32]]]


запуск алгоритма

In [51]:
CXPB, MUTPB, NGEN = 0.7, 0.2, 10

# Evaluate the entire population
fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

g=0

while (g <NGEN) :
    print('generation',g)
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop),tournsize=2)
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1[0], child2[0])
            
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    
    print(pop)
    fits_1 = [ind.fitness.values[0] for ind in pop]
    print(fits_1)
    fits_2 = [ind.fitness.values[1] for ind in pop]
    print(fits_2)
    
    g+=1

generation 0
[[5, 38]]
[[2, 38]]
[[[4, 33]], [[5, 34]], [[4, 33]], [[5, 40]], [[4, 34]], [[4, 38]], [[4, 36]], [[2, 38]], [[4, 36]], [[2, 42]]]
[1.4005217614785852, 0.6616446579027284, 1.3212626773804879, 0.7917446036573653, 1.5077299187147837, 1.0018196883206913, 1.157546163549895, 0.9863813533638404, 1.1182633476095818, 0.560732024551212]
[132.0, 170.0, 132.0, 200.0, 136.0, 152.0, 144.0, 76.0, 144.0, 84.0]
generation 1
[[5, 34]]
[[[4, 33]], [[4, 38]], [[4, 36]], [[4, 36]], [[4, 38]], [[4, 36]], [[5, 41]], [[2, 38]], [[4, 38]], [[2, 36]]]
[1.0327754602636632, 0.41303935689589244, 0.9136601987646182, 0.6147510712505466, 0.39563285077429217, 0.6101610300803615, 0.3736601333046167, 0.9863813533638404, 0.5178560684312925, 1.1202132097974429]
[132.0, 152.0, 144.0, 144.0, 152.0, 144.0, 205.0, 76.0, 152.0, 72.0]
generation 2
[[[5, 41]], [[2, 38]], [[4, 38]], [[4, 38]], [[4, 38]], [[4, 38]], [[5, 36]], [[4, 41]], [[4, 33]], [[4, 38]]]
[0.3736601333046167, 0.9863813533638404, 0.656572156062624